In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import re
import os
from PIL import Image, ImageOps
from tqdm import tqdm

path = Path('../input/amazon-ml-challenge-24/student_resource 3')

In [2]:
from shutil import copyfile
copyfile(src = path/'src/constants.py', dst = "../working/constants.py")
copyfile(src = path/'src/sanity.py', dst = "../working/sanity.py")
copyfile(src = path/'src/utils.py', dst = "../working/utils.py")

'../working/utils.py'

In [3]:
df = pd.read_csv(path/'dataset/train.csv', low_memory=False)
test_df = pd.read_csv(path/'dataset/test.csv', low_memory=False)
df

,image_link,group_id,entity_name,entity_value
0,https://m.media-amazon.com/images/I/61I9XdN6OF...,748919,item_weight,500.0 gram
1,https://m.media-amazon.com/images/I/71gSRbyXmo...,916768,item_volume,1.0 cup
2,https://m.media-amazon.com/images/I/61BZ4zrjZX...,459516,item_weight,0.709 gram
3,https://m.media-amazon.com/images/I/612mrlqiI4...,459516,item_weight,0.709 gram
4,https://m.media-amazon.com/images/I/617Tl40LOX...,731432,item_weight,1400 milligram
...,...,...,...,...
263854,https://m.media-amazon.com/images/I/612J1R1xHl...,558806,height,5.0 centimetre
263855,https://m.media-amazon.com/images/I/61Blzh2+28...,470067,height,8.5 inch
263856,https://m.media-amazon.com/images/I/51MsegDL9V...,204245,height,43.2 centimetre
263857,https://m.media-amazon.com/images/I/510KhVw4VS...,752266,height,9.1 centimetre


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263859 entries, 0 to 263858
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   image_link    263859 non-null  object
 1   group_id      263859 non-null  int64 
 2   entity_name   263859 non-null  object
 3   entity_value  263859 non-null  object
dtypes: int64(1), object(3)
memory usage: 8.1+ MB


In [6]:
from constants import entity_unit_map

allowed_units = entity_unit_map
unit_map = entity_unit_map
entity_unit_map

{'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
 'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
 'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
 'item_weight': {'gram',
  'kilogram',
  'microgram',
  'milligram',
  'ounce',
  'pound',
  'ton'},
 'maximum_weight_recommendation': {'gram',
  'kilogram',
  'microgram',
  'milligram',
  'ounce',
  'pound',
  'ton'},
 'voltage': {'kilovolt', 'millivolt', 'volt'},
 'wattage': {'kilowatt', 'watt'},
 'item_volume': {'centilitre',
  'cubic foot',
  'cubic inch',
  'cup',
  'decilitre',
  'fluid ounce',
  'gallon',
  'imperial gallon',
  'litre',
  'microlitre',
  'millilitre',
  'pint',
  'quart'}}

In [7]:
df.loc[:,'unit'] = df.entity_value.str.split(' ').str[-1]

filtered_df = df[~((df.entity_name=='item_weight')&(~df.unit.isin(allowed_units['item_weight'])))].copy()
filtered_df.reset_index(drop=True, inplace=True)

In [8]:
!mkdir '../working/images'
img_folder = Path('../working/images')

mkdir: cannot create directory '../working/images': File exists


In [9]:
import aiohttp
import asyncio
import nest_asyncio
from utils import create_placeholder_image

# Async function to download a single image
async def download_image(session, image_link, save_folder):
    if not isinstance(image_link, str):
        return

    filename = Path(image_link).name
    image_save_path = os.path.join(save_folder, filename)

    if os.path.exists(image_save_path):
        return
    try:
        async with session.get(image_link) as response:
            if response.status == 200:
                with open(image_save_path, 'wb') as f:
                    f.write(await response.read())
            else:
                create_placeholder_image(image_save_path)
    except:
        create_placeholder_image(image_save_path)

In [10]:
# Async function to download all images
async def download_images(image_links, download_folder):
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    async with aiohttp.ClientSession() as session:
        tasks = []
        for image_link in image_links:
            tasks.append(download_image(session, image_link, download_folder))
        
        for f in tqdm(asyncio.as_completed(tasks), total=len(tasks)):
            await f

nest_asyncio.apply()
# Main function to trigger the download
async def start_async_download(image_links, download_folder):
    await download_images(image_links, download_folder)

In [11]:
image_links = test_df.image_link.tolist()
await start_async_download(image_links, download_folder=img_folder)

100%|██████████| 131187/131187 [00:06<00:00, 19571.42it/s]


In [12]:
fnames = os.listdir(img_folder)
len(fnames)

90666

In [13]:
import easyocr

def load_images(image_paths):
    images = []
    for path in image_paths:
        try:
            image = Image.open(path).convert('L')
            image = image.resize((256,256), Image.Resampling.LANCZOS)
        except OSError: image = Image.new('RGB',(256,256),(255,0,0))
        binarized = ImageOps.autocontrast(image, cutoff=0)
        img = np.array(binarized)
        images.append(img)
    return images

reader = easyocr.Reader(['en'], gpu=True)
def extract_txt(image_paths):
    images = load_images(image_paths)
    results = reader.readtext_batched(images)
    text_list = []
    for result in results:
        text = ""
        for (_, txt, _) in result: text += ' '+txt
        text_list.append(text)
    return text_list

/opt/conda/lib/python3.10/site-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(copyStateDict(torch.load(trained_model, ma

In [14]:
def process_df(df, bs=64):
    for i in tqdm(range(0, len(df), bs), desc='Extracting text'):
        batch = df.iloc[i:i+bs]
        img_paths = [img_folder/Path(link).name for link in batch.image_link]
        df.loc[i:i+bs-1, 'extracted_text'] = extract_txt(img_paths)
    return df

In [15]:
processed_df = process_df(test_df, bs=64)

Extracting text: 100%|██████████| 2050/2050 [3:28:36<00:00,  6.11s/it]  


In [69]:
processed_df.to_csv('AMLC_text_extracted_file.csv', index=False)

In [31]:
def replace_chars(text):
    # Step 1: Replace all 'z' and 'Z' with '2'
    text = text.replace('z', '2').replace('Z', '2')
    
    # Step 2: Replace all 'S' and 's' with '5'
    text = text.replace('s', '5').replace('S', '5')
    
    # Step 3: Replace 'o' and 'O' with '0' if they appear after or before a digit
    text = re.sub(r'(?<=\d)[oO]|[oO](?=\d)', '0', text)
    
    return text

In [32]:
processed_df['extracted_text'] = processed_df['extracted_text'].apply(replace_chars)

In [21]:
wts = ['g','G','grams','Grams','kg','Kg','KG','mg','Mg','lbs','oz','OZ']
watts = ['w','kw','W','kW']
volt = ['kv','V','kv']
vol = ['L','mL','ml']

unit_map['item_weight'].update(wts)
unit_map['maximum_weight_recommendation'].update(wts)
unit_map['wattage'].update(watts)
unit_map['voltage'].update(volt)
unit_map['item_volume'].update(vol)

In [22]:
def build_pattern(units):
    unit_pattern = '|'.join(units)
    return rf"(\d+(\.\d+)?\s*({unit_pattern}))"

entity_patterns = {entity: build_pattern(units)
                   for entity, units in unit_map.items()}

def extract_entity_regex(text, entity_name):
    pattern = entity_patterns.get(entity_name)
    match = re.search(pattern, text)
    if match: return match.group(0)
    else: return ''

In [63]:
tqdm.pandas()
processed_df['prediction'] = processed_df.progress_apply(lambda row: extract_entity_regex(row['extracted_text'], row['entity_name']), axis=1)

100%|██████████| 131187/131187 [00:02<00:00, 44020.14it/s]


In [64]:
correct_unit = {'g':'gram', 'G':'gram', 'grams':'gram', 'Grams':'gram',
                'kg':'kilogram', 'Kg':'kilogram', 'KG':'kilogram',
                'mg':'milligram','Mg':'milligram', 'lbs':'pound',
                'oz':'ounce', 'OZ':'ounce', 'V':'volt', 'w':'watt',
                'kW':'kilowatt','W':'watt','kw':'kilowatt',
                'L':'litre','mL':'millilitre','ml':'millilitre'}

In [65]:
def format_value(pred):
    # Use regex to separate the numeric part from the unit part
    match = re.match(r"(\d+\.?\d*)\s*(.*)", pred)
    if match:
        number, unit = match.groups()
        number = float(number)
        # Standardize the unit using the mapping, default to the original unit if not in the mapping
        unit = correct_unit.get(unit, unit)
        # Return the formatted value
        return f"{number} {unit}"
    else:
        # If the format is unexpected, return the value as is
        return pred

In [66]:
processed_df['prediction'] = processed_df['prediction'].apply(format_value)

In [60]:
cond = (processed_df.extracted_text!='') & (processed_df.prediction!='')
processed_df.loc[cond, ['image_link','entity_name','extracted_text','prediction']]

,image_link,entity_name,extracted_text,prediction
18,https://m.media-amazon.com/images/I/218BCzgKxu...,wattage,Dax Rcd Gen 82WW Ha (Uhl L2 CH Dax Hk Gen,82.0 watt
74,https://m.media-amazon.com/images/I/31+1xiVjZB...,wattage,230mm Capacity: 1.4L Power: I500W J60mm,500.0 watt
82,https://m.media-amazon.com/images/I/31+NHijx4t...,item_weight,11.1lb / 5.05 kg 5uper Lightweight,5.05 kilogram
112,https://m.media-amazon.com/images/I/3101lgy28B...,wattage,3rt brt050w Jn,50.0 watt
147,https://m.media-amazon.com/images/I/310uxz1C1+...,item_volume,Bi3 LRARO DR 2 4It L38 38 =,3.0 litre
...,...,...,...,...
131122,https://m.media-amazon.com/images/I/91xjUO8Hel...,item_weight,TELK ADJU5TABLE DUMBBE2L5 GL055 FINI5H 1051e5 ',5.0 gram
131130,https://m.media-amazon.com/images/I/91xuRazRtQ...,item_weight,NORDIC LIFTING 5.4KG 12LB,5.4 kilogram
131167,https://m.media-amazon.com/images/I/A1MLm3ohGJ...,item_weight,100% 10g PXR BPTTEBFA5E] MaturAL Kontt,10.0 gram
131174,https://m.media-amazon.com/images/I/A1cAJrp-T5...,item_weight,"Ilkk McCormick"" ONE 5HEET PAN GLA2ED 5ALMON V...",5.0 gram


In [67]:
processed_df[['index','prediction']].to_csv('submission.csv', index=False)

In [68]:
from sanity import sanity_check

sanity_check(str(path/'dataset/test.csv'), '../working/submission.csv')

Parsing successfull for file: ../working/submission.csv


In [ ]:
def f1score(df, OUT='value_pred', GT='entity_value'):
    fp, tp, fn, tn = 0,0,0,0
    for _,row in df.iterrows():
        if (row[GT] == '')&(row[OUT] == ''): tn += 1
        elif (row[GT] != '')&(row[OUT] == ''): fn += 1
        elif (row[GT] == '')&(row[OUT] != ''): fp += 1
        elif row[GT] != row[OUT]: fp += 1
        else: tp += 1
            
    precision = tp/(tp+fp); recall = tp/(tp+fn)
    return 2*precision*recall/(precision+recall)

In [ ]:
# f1score(processed_df)